In [1]:
print("Training routine is loaded")

Training routine is loaded


In [1]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import torch


def train_model(model, train_dataloader, test_dataloader, loss_function, num_epochs=5, optimizer=None, device=None):
    """
    Universal model training function. 

    Inputs:
            model - torch.nn.Module
            dataloader
            loss
            num_epochs - number of epochs, default = 5
            optimizer - from torch.optim.Adam
    """
    
    if device is None:
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")    
      
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters())

    model = model.to(device)

    train_losses = []
    test_losses = []
    train_precision = []
    test_precision = []
    train_recall = []
    test_recall = []
    train_f1 = []
    test_f1 = []
    train_auc = []
    test_auc = []

    pbar = tqdm(range(num_epochs), total=num_epochs)

    for epoch in pbar:
        model.train()
        for batch_idx, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = loss_function(outputs, y.float())
            loss.backward()
            optimizer.step()

        train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            for batch_idx, (X, y) in enumerate(test_dataloader):
                X, y = X.to(device), y.to(device)
                outputs_test = model(X)
                test_loss = loss_function(outputs_test, y.float())
                
            test_losses.append(test_loss.item())

        pbar.set_description(f"Epoch: {epoch+1}, Train Loss: {train_losses[-1]}, Test Loss: {test_losses[-1]}")

    # Plotting losses
    plt.figure(figsize=(10, 7))
    plt.plot(train_losses, color='blue', label='train loss')
    plt.plot(test_losses, color='red', label='test loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plotting metrics
    plt.figure(figsize=(10, 7))
    plt.plot(train_precision, color='blue', label='train precision')
    plt.plot(test_precision, color='red', label='test precision')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.legend()
    plt.show()
    
        # Plotting recall
    plt.figure(figsize=(10, 7))
    plt.plot(train_recall, color='blue', label='train recall')
    plt.plot(test_recall, color='red', label='test recall')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    plt.legend()
    plt.show()

    # Plotting f1 score
    plt.figure(figsize=(10, 7))
    plt.plot(train_f1, color='blue', label='train f1')
    plt.plot(test_f1, color='red', label='test f1')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.show()

    # Plotting AUC
    plt.figure(figsize=(10, 7))
    plt.plot(train_auc, color='blue', label='train auc')
    plt.plot(test_auc, color='red', label='test auc')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    plt.show()

    return model


In [18]:
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_auc_score

def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float())  # Remove .view(-1, 1) from labels
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}")

    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(torch.sigmoid(outputs).cpu().numpy().flatten())
        score = roc_auc_score(y_true, y_pred)
        print(f"ROC/AUC Score: {score:.4f}")

    return score